# A simple implementation of Shor's algorithm

This notebook shows how to implement Shor's algorithm using myQLM.


## The high-level routine.


We suppose we have access to the following python functions:

- ``is_prime(N)`` that tells us if an integer $N$ is a prime number of not
- ``compute_prime_power(N)`` that returns the prime number $u$ such that $u^b = N$, if it exists, and -1 otherwise
- ``compute_order(a, N, n_phase_bits, qpu)`` computes the order $r$ of the function $f(x) = a^x\,\mathrm{mod.}\, N$, i.e the smallest integer $r$ such that $a^r = 1 \,\mathrm{mod.}\, N$. This order is computed using a phase estimation algorithm (hence the arguments ``n_phase_bits`` and ``qpu``).

We first write the high-level function using these subfunctions to compute the factorization of an integer $N$:

In [ ]:
def perform_shor(N, qpu, max_n_trials=100, n_phase_bits=None, nbshots=100, verbose=False):
    """This is the main routine to perform Shor's algorithm.
    
    
    Args:
        N (integer): the number to be factorized
        qpu (QPU): a quantum processing unit 
        max_n_trials (int, optional): the maximum number of trials of "a" before giving up.
            Defaults to 100. It should in principle not be reached.
        n_phase_bits (int): number of phase bits to be used in quantum phase estimation.
        nbshots (int, optional): the number of shots (repetitions). Defaults to 100.
        verbose (bool, optional): for verbose output. Defaults to False.
    """
    # handle trivial cases: N even, prime, or power of prime
    if N%2 == 0: return 2
    if is_prime(N): return N
    u = compute_prime_power(N)
    if u != -1: return u
    
    already_tested = []
    n_it = 0
    while n_it < max_n_trials:
        n_it += 1
        a = random.randint(3, N)
        if a in already_tested:
            continue
        else:
            already_tested.append(a)
        if verbose:
            print("Trying with a = %s"%a)
            
        # handle trivial cases
        g = math.gcd(a, N)
        if g != 1:
            return g
        
        # else: a and N are coprime
        # this is where the QPU is called!
        r = compute_order(a, N, n_phase_bits, qpu, nbshots, verbose)
        if verbose:
            print("Found order r = %s"%r)
            
        if r % 2 == 1:
            if verbose:
                print("r odd, trying new a...")
            continue
        if (a**(r//2)) % N == -1:
            if verbose:
                print("(a^(r//2)) % N == -1, trying new a...")
            continue
        g1 = math.gcd(a**(r//2)+1, N)
        g2 = math.gcd(a**(r//2)-1, N)
        if N % g1 == 0 and g1 != N: return g1
        if N % g2 == 0 and g2 != N: return g2
        
        
    raise Exception("Did not find factorization, try increasing max_n_trials!")

## A few arithmetic subroutines

We now implement the arithmetic subroutines ``is_prime`` and ``compute_prime_power``:

In [ ]:
import math
def is_prime(N):
    """Function that checks whether a number is prime
    
    Args:
        N (int): an integer
    Returns:
        bool: True if N is prime
    """
    for i in range(2, N-1):
        if N % i == 0:
            return False
    else:
        return True
    
def compute_prime_power(N):
    """
    O(L^3) algo to check if N is the power of a prime number
    
    Returns:
        int: u s.t u**b == N if exists, else -1
    """
    L = math.ceil(math.log2(N))
    for b in range(2, L):
        u1 = math.floor(N**(1./b))
        u2 = u1 + 1
        if u1**b == N: return u1
        if u2**b == N: return u2
    return -1

# checks
assert(compute_prime_power(13**4) == 169)
assert(compute_prime_power(11*3) == -1)

## The quantum part

We now turn to the quantum part of the algorithm. It is performed using the ``compute_order`` function, which essentially consists of two steps:

- a **quantum phase estimation step**, which returns an approximation to $k/r$, with $r$ the period of $f(x)$ as defined above, and $k\in\mathbb{N}$. This is done by a function ``perform_shor_qpe(a, N, n_phase_bits, qpu)`` that executes and processes the results of the QPE quantum circuit corresponding to Shor's problem (the circuit itself it produced by a function ``make_shor_circ(a, N, n_phase_bits)``.

- a **continued fraction expansion step**, which tries to extract $r$ from the approximation to $k/r$ (it does not always succeed!!). It is a classical step (despite the title of the section).


Here is the QPE circuit:
![Shor circuit](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6b/Shor%27s_algorithm.svg/1920px-Shor%27s_algorithm.svg.png)

Note that the controlled unitaries of the QPE correspond, **when put together**, to a "modular exponentiation" operation (see the documentation of myqlm's ``modular_exp`` function).

Let us first implement the ``compute_order`` function, assuming we have access to ``perform_shor_qpe``. For the continued fraction expansion, we use the ``fractions`` Python module (see ``limit_denominator``).

In [ ]:
import numpy as np
from fractions import Fraction
        
def compute_order(a, N, n_phase_bits, qpu, nbshots, verbose=False):
    """
    Find lowest positive integer r such that a^r = 1 mod. N
    
    Args:
        a (int): the integer a
        N (int): the integer N
        n_phase_bits (int): number of phase bits for quantum phase estimation
        qpu (QPU): a quantum processing unit 
        nbshots (int, optional): the number of shots (repetitions). Defaults to 100.
        verbose (bool, optional): for verbose output. Defaults to False.
    """
    int_list = perform_shor_qpe(a, N, n_phase_bits, qpu, nbshots, verbose=verbose)
    
    if verbose:
        print("QPE yields the following list of integers = %s"%int_list)
    
    # try performing continued fraction expansion for the most probable output states
    for state_int in int_list:
        if verbose:
            print("Trying to find r for state_int = %s"%state_int)
            
        ## do continued fraction expansion on state_int
        # ...
        # return r
    
    raise Exception("Could not compute order. Try increasing nbshots.")

We now implement ``perform_shor_qpe``, which uses ``make_shor_circ``

In [ ]:
import random
from qat.lang.AQASM.qftarith import IQFT
from qat.lang.AQASM import Program, H, X
from qat.lang.AQASM.arithmetic import modular_exp
import qat.lang.AQASM.qftarith as qftarith
import qat.lang.AQASM.classarith as classarith

from qat.core.util import statistics

def make_shor_circ(a, N, n_phase_bits):
    """Construct the circuit corresponding to the quantum part
    of Shor's algorithm, namely the quantum phase estimation routine
    that is used to find the solution r to the equation
    
    .. math::
        a^r = 1 mod. N
    
    Args:
        a (integer): the integer a
        N (integer): the integer N
        n_phase_bits (int): the number of phase bits
    
    Returns:
        Circuit: the quantum circuit
    """
    L = math.ceil(math.log2(N))  # number of data qubits
    
    prog = Program()
    phase_reg = prog.qalloc(n_phase_bits)
    data_reg = prog.qalloc(L)
   
    # ...

    
    #return prog.to_circ(link=[classarith])
    return prog.to_circ(link=[qftarith])

def perform_shor_qpe(a, N, n_phase_bits, qpu, nbshots=100, verbose=False):
    """This function performs and post-processes the quantum
    phase estimation of Shor's algorithm.
    
    Args:
        a (integer): the integer a
        N (integer): the integer N
        n_phase_bits (int): the number of phase bits
        qpu (QPU): a qpu
        nbshots (int, optional): the number of shots (repetitions). Defaults to 100.
        verbose (bool, optional): for verbose output. Defaults to False.
        
    Returns:
        list<int>: the nbshots-long list of integers (betw 0 and 2**n_phase_bits - 1)
            from which the period r can be extracted
    """
    circ = make_shor_circ(a, N, n_phase_bits)
    
    # ... execute on QPU and return list of outputs (sorted by decreasing prob)
    
    return int_list_sorted
        
    raise Exception("Did not get any sample different from zero. Should increase nbshots")

## A peek into the quantum routine

Let us take a look at the final distribution produced by ``make_shor_circ`` for $N=15$ and a trial $a=13$. 

1) For this case, show (numerically) that the period is $r=4$.

2) Let us now run the circuit and plot its distribution:

In [ ]:
%%time
from qat.qpus import PyLinalg
N = 15
a = 13
n_phase_bits = 5
circ = make_shor_circ(a, N, n_phase_bits)
   
r = 4 # for a = 13, N = 15

# compute probs_out: distribution of 2**n_phase_bits probabilities
    
import matplotlib.pyplot as plt
plt.plot(psi_out)
expect = [2**n_phase_bits * k / r for k in range(7)]
plt.plot(expect, [0.02 for _ in expect], 'sr')

## Running the full Shor algorithm

Now, you can launch ``perform_shor`` for $N=15$. What do you see? If you relaunch it, do you observe the same behavior?

In [ ]:
%%time
from qat.qpus import PyLinalg

N = 15
n_phase_bits = 5

qpu = PyLinalg()

res = perform_shor(N, qpu, max_n_trials=100, n_phase_bits=n_phase_bits, verbose=True)
print("a factor of N = %s is %s"%(N, res))

## Further questions

- What happens when ``n_phase_bits`` becomes larger?
- What if you were to use a "noisy" QPU (using the ``DepolarizingPlugin`` of previous lecture)? (Warning: it may take much more run time to get the final result)